## **내 드라이브 연결** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/myModel

In [ ]:
# 현재 경로 확인
pwd

##**사용할 라이브러리 추가**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
# 코랩에서는 외부에 다운받은 라이브러리 import 못하는듯
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## **데이터 불러오기**

In [ ]:
batch_size = 16
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize((64, 64))
])

train_data = torchvision.datasets.ImageFolder('./train', transform=transform)
test_data = torchvision.datasets.ImageFolder('./test', transform=transform)

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
print(train_data)
print(test_data)

## **데이터 이미지화**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imgshow(img):
    # img = img / 2 + 0.5 # 이미지를 흐려지게 만드려고 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # 3차원 데이터라서 transpose할 때 shape 지정해줘야됨
    plt.show()

dataiter = iter(train_loader) # 원하는 데이터 넣기
img, lab = dataiter.next()
print(img.shape)

imgshow(torchvision.utils.make_grid(img))

classes = ('reja1', 'reja2', 'reja3', 'reja4', 'reja5', 'reja6', 'reja7', 'reja8', 'reja9', 'silicon1', 'silicon2', 'silicon3', 'silicon4', 'silicon5', 'silicon6', 'silicon7', 'silicon8', 'silicon9')

# label 출력
print(' '.join('%s' % classes[lab[i]] for i in range(batch_size)))

## **모델 만들기**

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            # nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2),

            nn.Conv2d(16, 32, 5),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, 5),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 12 * 12, 100), # shape 계산해서 첫번째 인자 맞춰주기 !!
            nn.BatchNorm1d(100), # 2d 하면 에러나씀
            nn.LeakyReLU(0.2),
            nn.Linear(100, 18) # class 개수로 맞춰주기
        )
    
    def forward(self, x):
        out = self.layer(x) # out 형태 (batch_size, channel, row_size, col_size)
        out = out.view(out.size(0), -1) # Linear에 들어가기 전 shape 변경
        out = self.fc_layer(out) 
        return out

### **장치설정** ( GPU / CPU 환경 설정 )

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### **모델을 장치로 보내주기**

In [ ]:
net = CNN().to(device)
net # 모델 정보 출력됨

## **모델 shape 확인**

In [ ]:
pip install pytorch-model-summary

In [ ]:
# 모델 shape 출력
import pytorch_model_summary as summary
print(summary.summary(net, torch.zeros(batch_size, 3, 64, 64).to(device), show_input = True))
print(summary.summary(net, torch.zeros(batch_size, 3, 64, 64).to(device), show_input = False))

## **Train**

### **train 환경 설정**

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)
loss_func = nn.CrossEntropyLoss().to(device)
epochs = 100
patience = 10
data_loader = train_loader

### **학습 시작**

In [ ]:
total_batch = len(data_loader) # 한 epoch 당 minibatch 개수

for epoch in range(epochs):
    net.train()
    early_stopping = EarlyStopping(patience = 20, verbose = False)
    avg_cost = 0.0     
    for i, (inputs, labels) in enumerate(data_loader):
      # inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = net(inputs)
      loss = loss_func(outputs, labels)
      loss.backward()
      optimizer.step()
        
      avg_cost += loss / total_batch

    early_stopping(avg_cost, net)
    if early_stopping.early_stop:
      print("early stop !!")
      break

    print('[Epoch:{}/{}] cost = {:.6f}'.format(epoch+1, epochs, avg_cost))
print('Learning Finished!')

## **Test**

### **데이터 예측하기**

In [ ]:
net.eval()
# classes = ('reja1', 'reja7', 'reja9', 'silicon1', 'silicon7', 'silicon9')
classes = ('reja1', 'reja2', 'reja3', 'reja4', 'reja5', 'reja6', 'reja7', 'reja8', 'reja9', 'silicon1', 'silicon2', 'silicon3', 'silicon4', 'silicon5', 'silicon6', 'silicon7', 'silicon8', 'silicon9')

# 이미지 가져오기
dataiter = iter(test_loader)
img, lab = dataiter.next() # 배치 1개 가져오기, 설정한 배치 사이즈의 개수만큼 가져옴

imgshow(torchvision.utils.make_grid(img)) # 이미지로 확인

print('<original label>')
print(' '.join('%s' % classes[lab[i]] for i in range(batch_size))) # label 확인

# 예측하기
img, lab = img.to(device), lab.to(device)
outputs = net(img)
_, predict = torch.max(outputs, 1)

print('<predicted label>')
print(' '.join('%s' % classes[predict[i]] for i in range(batch_size)))


### **testset accuracy 출력**

In [ ]:
print(len(test_loader.dataset))
print(len(test_loader))
correct = 0
total = 0
test_loss = 0
import matplotlib.pyplot as plt

lst = []
with torch.no_grad():
    for idx, data in enumerate(test_loader):
        net.eval()
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)

        # reduction = 'sum'을 입력해서 미니배치의 합을 받아옴
        # 오차의 합 구하기
        test_loss += F.cross_entropy(outputs, labels, reduction='sum').item()
        test_loss /= len(test_loader.dataset)

        _, predicted = torch.max(outputs.data, 1) # 행방향 최대값 // predicted: 최대값 인덱스가 담긴 리스트
        # predicted = output.max(1, keepdim=True)[1] 이거랑 같음

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(correct)
        lst.append(100 * correct / total)
        # plt.plot(idx, idx)
        # plt.show()
    
        print('[{}/{}] Accuracy: {:.2f}% Test Loss: {:.4f}'.format(idx, len(test_loader), 100 * correct / total, test_loss))

plt.plot([0,1,2,3,4], lst)

### **class 별 testset accuracy 출력**

In [ ]:
correct_pred = {classname: 0 for classname in classes} # 0으로 초기화 {'plane': 0, 'car': 0, 'bird': 0, 'cat': 0, 'deer': 0, 'dog': 0, 'frog': 0, 'horse': 0, 'ship': 0, 'truck': 0}
total_pred = {classname: 0 for classname in classes}
# print('correct_pred:', correct_pred)
# print('total_pred:', total_pred)

with torch.no_grad():
    for idx, data in enumerate(test_loader):
        net.eval()
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        # outputs.data 랑 outputs 결과 똑같음
        _, predicted = torch.max(outputs.data, 1)
        # print('pre:', predicted)
        for label, prediction in zip(labels, predicted):
          if label == prediction:
            correct_pred[classes[label]] += 1
          total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
  accuracy = 100 * float(correct_count) / total_pred[classname]
  print('Accuracy for class [{:s}] is: {:.1f}%'.format(classname, accuracy)) 

## **모델 저장**

In [ ]:
# 전체 모델 저장
# 모델 파라미터, 옵티마이저, 에포크 등 ...
path = './yuminModel.pt'
net = CNN()
torch.save(net, path)

In [ ]:
# 모델의 매개변수만 저장
path = './yuminModel.pth'
torch.save(net.state_dict(), path)

## **모델 불러오기**

In [ ]:
# 저장한 모델 불러오기
net = CNN()
path = './yuminModel.pth'
net.load_state_dict(torch.load(path))

# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# # 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])